In [1]:
import json, jsonschema, jwt, datetime

In [2]:
def mint_did(uuid):
    return f'did:merit:{uuid}'

In [3]:
merit = json.load(open("test_merit.json", "r"))
merit

{'id': 'dc30880d-2d4b-447c-905e-b79075aea95b',
 'acceptedAt': '2023-07-17T08:52:19.489566Z',
 'active': True,
 'activenessFailures': None,
 'authorizedAt': '2023-07-17T08:52:15.960677Z',
 'completed': True,
 'completenessFailures': None,
 'createdAt': '2023-07-17T15:52:04.3018Z',
 'description': 'Template for admin merit',
 'fields': [{'canShare': False,
   'description': "Use this to store recipient's first name",
   'fieldKind': {'fieldType': 'Text',
    'id': 'a9fa2832-4107-4c95-b7d8-0cdd15a7e2bc',
    'name': 'Text'},
   'name': 'First Name',
   'templateFieldID': '1b397a27-14a0-4d43-a36f-6006611f4073',
   'templateFieldLineage': ['3dc3f769-70cc-4e7e-81cc-ed103f9eb601'],
   'updatedAt': '2023-07-17T15:52:19.496539Z',
   'validationErrors': None,
   'value': 'Hank'},
  {'canShare': False,
   'description': "Use this to store recipient's last name",
   'fieldKind': {'fieldType': 'Text',
    'id': 'a9fa2832-4107-4c95-b7d8-0cdd15a7e2bc',
    'name': 'Text'},
   'name': 'Last Name',
   

In [4]:
verifiable_credential = {}

## Contexts

In [5]:
verifiable_credential["@context"] = [
    "https://www.w3.org/2018/credentials/v1",
    "https://www.w3.org/2018/credentials/examples/v1",
]

## Identifiers

In [6]:
credential = mint_did(merit["id"])
issuer = mint_did(merit["issuer"]["id"])
subject = mint_did(merit["recipient"]["id"])

verifiable_credential["issuer"] = issuer
verifiable_credential["id"] = credential

## Types

In [7]:
verifiable_credential["type"] = ["VerifiableCredential", merit["name"]]

## Credential Subject

In [8]:
verifiable_credential["credentialSubject"] = { 
    "id": subject,
    "merit": { field["name"]: field["value"] for field in merit["fields"] },
}

## Issuance date

In [9]:
verifiable_credential["issuanceDate"] = merit["createdAt"]

## Credential Status

In [10]:
verifiable_credential["credentialStatus"] = {
    "id": "https://api.merit.com/status?foo=fum",
    "type": "MeritCredentialStatusList2024",
    "statusPurpose": merit['state']['name'],
    "occurredAt": merit['state']['occurredAt']
}

## Validation

In [11]:
schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "@context": {
            "type": "array",
            "items": {
                "type": "string"
            },
            "minItems": 1
        },
        "id": {
            "type": "string"
        },
        "type": {
            "type": "array",
            "items": {
                "type": "string"
            },
            "contains": {
                "const": "VerifiableCredential"
            }
        },
        "issuer": {
            "type": "string"
        },
        "issuanceDate": {
            "type": "string",
            "format": "date-time"
        },
        "credentialSubject": {
            "type": "object",
            "properties": {
                "id": {
                    "type": "string"
                }
            },
            "required": ["id"]
        },
        "credentialStatus": {
            "type": "object",
            "properties": {
                "id": {
                    "type": "string"
                },
                "type": {
                    "type": "string"
                },
                "statusPurpose": {
                    "type": "string"
                },
                "occurredAt": {
                    "type": "string",
                    "format": "date-time"
                }
            },
            "required": ["id"]
        },
        "proof": {
            "type": "object",
            "properties": {
                "type": {
                    "type": "string"
                },
                "created": {
                    "type": "string",
                    "format": "date-time"
                },
                "proofValue": {
                    "type": "string"
                },
                "verificationMethod": {
                    "type": "string"
                },
                "jws": {
                    "type": "string"
                }
            },
            "required": ["type", "created", "proofValue"]
        }
    },
    "required": ["@context", "type", "issuer", "issuanceDate", "credentialSubject"]#, "proof"]
}

try:
    jsonschema.validate(instance=verifiable_credential, schema=schema)
    print("The Verifiable Credential is valid.")
except jsonschema.exceptions.ValidationError as e:
    print("The Verifiable Credential is not valid:", e)

The Verifiable Credential is valid.


## Proofs

In [12]:
payload = {
    "iss": issuer,  # Issuer
    "sub": subject,  # Subject
    "aud": "https://verifier.example.gov",  # Audience
    "exp": datetime.datetime.utcnow() + datetime.timedelta(days=1),  # Expiration time
    "nbf": datetime.datetime.utcnow(),  # Not before
    "iat": datetime.datetime.utcnow(),  # Issued at
    "vc": verifiable_credential  # The Verifiable Credential
}
secret_key = 'secret'
jwt_token = jwt.encode(payload, secret_key, algorithm='HS256')
jwt_token


'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJkaWQ6bWVyaXQ6OWE1ZWI5MmEtYjM0YS00MmI5LThmYjctMTBjZTk0NDllODNlIiwic3ViIjoiZGlkOm1lcml0OjE5NmE4YzhmLTZmZDktNDU2Yi04MTkzLTQxNTliOGRjM2FiNiIsImF1ZCI6Imh0dHBzOi8vdmVyaWZpZXIuZXhhbXBsZS5nb3YiLCJleHAiOjE2OTQ4OTgzODgsIm5iZiI6MTY5NDgxMTk4OCwiaWF0IjoxNjk0ODExOTg4LCJ2YyI6eyJAY29udGV4dCI6WyJodHRwczovL3d3dy53My5vcmcvMjAxOC9jcmVkZW50aWFscy92MSIsImh0dHBzOi8vd3d3LnczLm9yZy8yMDE4L2NyZWRlbnRpYWxzL2V4YW1wbGVzL3YxIl0sImlzc3VlciI6ImRpZDptZXJpdDo5YTVlYjkyYS1iMzRhLTQyYjktOGZiNy0xMGNlOTQ0OWU4M2UiLCJpZCI6ImRpZDptZXJpdDpkYzMwODgwZC0yZDRiLTQ0N2MtOTA1ZS1iNzkwNzVhZWE5NWIiLCJ0eXBlIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiQWRtaW4gbWVyaXQiXSwiY3JlZGVudGlhbFN1YmplY3QiOnsiaWQiOiJkaWQ6bWVyaXQ6MTk2YThjOGYtNmZkOS00NTZiLTgxOTMtNDE1OWI4ZGMzYWI2IiwibWVyaXQiOnsiRmlyc3QgTmFtZSI6IkhhbmsiLCJMYXN0IE5hbWUiOiJMaW5kZ3JlbiIsIkVtYWlsIjoib2xlZyt1c2VyMkBnb21lcml0cy5jb20iLCJJc3N1aW5nIE9yZyBOYW1lIjoiTWVyaXQgSW50ZXJuYXRpb25hbCwgSW5jLiIsIk9yZyBVVUlEIjoidXNlcjIiLCJBZG1pbiBQaG9uZSBOdW1iZXIiOiI2MzE3MD

## Verification

In [13]:
decoded_payload = jwt.decode(jwt_token, secret_key, algorithms='HS256', audience='https://verifier.example.gov')
decoded_payload['vc']

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1'],
 'issuer': 'did:merit:9a5eb92a-b34a-42b9-8fb7-10ce9449e83e',
 'id': 'did:merit:dc30880d-2d4b-447c-905e-b79075aea95b',
 'type': ['VerifiableCredential', 'Admin merit'],
 'credentialSubject': {'id': 'did:merit:196a8c8f-6fd9-456b-8193-4159b8dc3ab6',
  'merit': {'First Name': 'Hank',
   'Last Name': 'Lindgren',
   'Email': 'oleg+user2@gomerits.com',
   'Issuing Org Name': 'Merit International, Inc.',
   'Org UUID': 'user2',
   'Admin Phone Number': '6317070926',
   'Org Legal Name': 'Org legal name 74d00472-85d5-4c6b-81d3-5b6353a8a7fd',
   'Org Name': 'Organization Name b6e9f27b-6214-4f57-a670-5e02d2a1fdeb',
   'Org Description': 'This is very important organization',
   'Org Governing Country': 'US',
   'Org Governing State': 'CA'}},
 'issuanceDate': '2023-07-17T15:52:04.3018Z',
 'credentialStatus': {'id': 'https://api.merit.com/status?foo=fum',
  'type': 'MeritCredentialStatusLis